## Splice the reference sequence for CD-search

The script splits the reference fasta file into segments of __~180,000__ nucleotides. This is the longest nucleotide length that can be inputted into __NCBI CD-search__. If the splice site would split the sequence within a gene, the coordinates are adjusted accordingly to keep all genes intact.

In [21]:
from Bio import SeqIO
import pandas as pd
import gffpandas.gffpandas as gffpd

First, read in the corresponding reference fasta and gene assembly GFF3, depending on the _Mycobacterium_ species.

In [22]:
# For Mycobacterium bovis:
species_name = 'M.bovis'
fasta_file = '../data/Mycobacterium bovis/Mbovis_AF2122_97 reference sequence.fasta'
gene_assembly_file = '../data/Mycobacterium bovis/LT708304 gene assembly.gff3'
output_folder = '../data/Mycobacterium bovis/Reference fasta segments for CD-search'

In [23]:
# For Mycobacterium tuberculosis:
species_name = 'M.tuberculosis'
fasta_file = '../data/Mycobacterium tuberculosis/NC_018143.1 reference sequence.fasta'
gene_assembly_file = '../data/Mycobacterium tuberculosis/NC_018143.1 gene assembly updated.gff3'
output_folder = '../data/Mycobacterium tuberculosis/Reference fasta segments for CD-search'

Isolate the reference sequence.

In [24]:
# Read in the reference sequence
reference_seq = SeqIO.read(open(fasta_file), 'fasta')
whole_seq = reference_seq.seq

Use the gene assembly file to find the locations of all the genes:

In [25]:
# Read in the gene assembly
gene_gff = gffpd.read_gff3(gene_assembly_file)
gene_df = gene_gff.df

In [26]:
# Store the gene co-ordinates in a list of tuples

gene_coords = []

for index, row in gene_df.iterrows():
    gene_coords.append((row['start'], row['end']))

Find the co-ordinates where the fasta file should be cut. This is approximately every 180,000 nucleotides. If this falls within a gene, iteratively remove nucleotides until it no longer does.

In [27]:
# Function to check if a position is within a gene
def is_within_gene(position, gene_coords):
    for start, end in gene_coords:
        if start <= position <= end:
            return True
    return False

In [28]:
# Start at 1 and look at the coordinate positioned 18,0000 nucleotides onwards.
# If it is not within a gene, that is the next starting co-ordinate and the cycle continues.
# If the position is within a gene, iteratively remove nucleotides until it is not. This is the next
# starting co-ordinate and the cylce continues up to the length of the fasta file.

start = 1
gap = 180000
cut_coords = []
len_seq = len(whole_seq)

# Up to the total length of the sequence...
while start < len_seq:
    # Calculate end splice coord
    end = start + gap
    # Check end coord isn't within a gene
    while is_within_gene(end, gene_coords):
        # Keep going down 1 position until no longer in gene
        end -= 1
    # If the end will be greater the sequence length
    if end > len_seq:
        # End max is the sequence length
        end = len_seq
    # Add start and end coords to cut_coords list
    cut_coords.append((start, end))
    # The next start coord begins at the following position
    start = end + 1

Use the calculated coordinates to splice the reference sequence into ~180,000 nucleotide chunks.

In [29]:
cut_seq = []

for coord in cut_coords:
    start, end = coord
    seq = whole_seq[start-1:end]
    cut_seq.append(seq)

Create a fasta file for each seperate sequence splice.

In [30]:
counter = 1

for index, seq in enumerate(cut_coords):
    # Modify the name of each fasta file with the segment coordinates
    name = f"{output_folder}/{species_name} section {counter} - {cut_coords[index][0]}:{cut_coords[index][1]}.fasta"
    fasta_header = f"> {species_name} section {counter} - {cut_coords[index][0]}:{cut_coords[index][1]}"
    
    # Find corresponding sequence segment
    seq = str(cut_seq[index])

    # Write fasta file
    with open(name, "w") as f:
        f.write(fasta_header + "\n")
        f.write(seq + "\n")

    counter += 1